# Quick Start

Загружаем нужные библиотеки.

In [1]:
import requests
import pandas as pd

from opendota import(
    query_opendota,
)

SQL-запрос к базе OpenDota: https://www.opendota.com/explorer

P.S. Если сначала выбрать что-то в полях, а затем нажать Toggle SQL, то автоматически сформируется нужный запрос

Схема БД: https://github.com/odota/core/blob/master/sql/create_tables.sql

In [2]:
ODOTA_QUERY = ''' 
SELECT
    matches.*,
    player_matches.*,
    ((player_matches.player_slot < 128) = matches.radiant_win) win,
    leagues.name leaguename
FROM 
    matches
        JOIN
            match_patch using(match_id)
        JOIN 
            leagues using(leagueid)
        JOIN 
            player_matches using(match_id)
        JOIN 
            heroes on heroes.id = player_matches.hero_id
        LEFT JOIN
            notable_players ON notable_players.account_id = player_matches.account_id AND notable_players.locked_until = (SELECT MAX(locked_until) FROM notable_players)
        LEFT JOIN 
            teams using(team_id)
WHERE TRUE
ORDER BY matches.match_id DESC NULLS LAST
LIMIT 200
'''

Используем функцию query_opendota, которая принимает на вход запрос и возвращает pandas.DataFrame()

In [3]:
df_matches = query_opendota(ODOTA_QUERY)
df_matches.head()

,ability_upgrades_arr,ability_uses,account_id,actions,additional_units,assists,backpack_0,backpack_1,backpack_2,barracks_status_dire,...,times,tower_damage,tower_status_dire,tower_status_radiant,towers_killed,version,win,xp_per_min,xp_reasons,xp_t
0,"[5155, 5154, 5154, 5155, 5154, 5157, 5154, 515...","{'sniper_shrapnel': 21, 'sniper_assassinate': 5}",113372833,"{'1': 4440, '2': 75, '3': 23, '4': 856, '5': 2...",None,4,0,0,0,63,...,"[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...",551,1982,1574,1.0,21.0,False,372,"{'0': 159, '1': 1030, '2': 7427}","[0, 145, 658, 1112, 1483, 1753, 2292, 2555, 27..."
1,"[5069, 5072, 5069, 5071, 5069, 5073, 5069, 507...","{'puck_illusory_orb': 43, 'puck_ethereal_jaunt...",88826062,"{'1': 1924, '3': 4, '4': 483, '5': 88, '6': 7,...",None,7,0,0,0,63,...,"[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...",914,1982,1574,0.0,21.0,False,504,"{'0': 276, '1': 3786, '2': 7612}","[0, 240, 740, 1210, 1798, 2411, 2984, 3373, 38..."
2,"[5595, 5596, 5595, 5597, 5595, 5598, 5595, 559...","{'legion_commander_overwhelming_odds': 31, 'le...",91932652,"{'1': 3707, '2': 10, '3': 109, '4': 619, '5': ...",None,8,11,0,0,63,...,"[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...",532,1982,1574,1.0,21.0,False,434,"{'0': 560, '1': 1689, '2': 7803}","[0, 57, 258, 535, 673, 987, 1323, 1701, 1939, ..."
3,"[5102, 5104, 5102, 5103, 5102, 5105, 5102, 510...","{'sandking_burrowstrike': 23, 'sandking_sand_s...",66620961,"{'1': 4021, '2': 48, '4': 206, '5': 26, '6': 1...",None,7,0,0,0,63,...,"[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...",0,1982,1574,0.0,21.0,False,328,"{'0': 456, '1': 3025, '2': 4108}","[0, 0, 187, 289, 360, 463, 616, 902, 1092, 125..."
4,"[5458, 5459, 5459, 5460, 5459, 5461, 5459, 546...","{'disruptor_thunder_strike': 17, 'disruptor_gl...",91355855,"{'1': 6505, '2': 98, '3': 10, '4': 466, '5': 4...",None,7,0,188,44,63,...,"[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...",0,1982,1574,0.0,21.0,False,220,"{'0': 859, '1': 1533, '2': 2717}","[0, 0, 63, 130, 336, 435, 463, 631, 694, 830, ..."


Смотрим что там в интересного в данных

In [4]:
df_matches.columns

Index(['ability_upgrades_arr', 'ability_uses', 'account_id', 'actions',
       'additional_units', 'assists', 'backpack_0', 'backpack_1', 'backpack_2',
       'barracks_status_dire',
       ...
       'times', 'tower_damage', 'tower_status_dire', 'tower_status_radiant',
       'towers_killed', 'version', 'win', 'xp_per_min', 'xp_reasons', 'xp_t'],
      dtype='object', length=118)

Найти нужного игрока можно по account_id

In [5]:
account_id = 20421681
dotabuff_player = 'https://www.dotabuff.com/players/'
opendota_player = 'https://www.opendota.com/players/'
dotabuff_url = '{}{}'.format(dotabuff_player, account_id)
opendota_url = '{}{}'.format(opendota_player, account_id)
print(dotabuff_url)
print(opendota_url)

https://www.dotabuff.com/players/20421681
https://www.opendota.com/players/20421681


Матч можно искать по match_id

In [6]:
match_id = 3458810313
dotabuff_match = 'https://www.dotabuff.com/matches/'
opendota_match = 'https://www.opendota.com/matches/'
db_url = '{}{}'.format(dotabuff_match, match_id)
odota_url = '{}{}'.format(opendota_match, match_id)
print(db_url)
print(odota_url)

https://www.dotabuff.com/matches/3458810313
https://www.opendota.com/matches/3458810313


Внутри матча каждый игрок сидит на одном из 10 слотов.

У нормальных людей слоты изменяются от 0 до 9,

У opendota слоты в рамках одного матча изменяются от 0 до 4 - The Radiant, а далее от 128 до 132 - The Dire

In [7]:
df_matches[['account_id', 'match_id', 'player_slot']].head(10)

,account_id,match_id,player_slot
0,113372833,3575051322,0
1,88826062,3575051322,1
2,91932652,3575051322,2
3,66620961,3575051322,3
4,91355855,3575051322,4
5,113331514,3575051322,128
6,92847434,3575051322,129
7,11550182,3575051322,130
8,5150808,3575051322,131
9,89269794,3575051322,132


Посчитаем кто больше всех нанес урона по вышкам в одном матче

In [20]:
df_best_tower_damage = (
    df_matches
    .sort_values('tower_damage', ascending=False)
    [['tower_damage', 'match_id', 'account_id', 'player_slot']]
    .reset_index(drop=True)
)
df_best_tower_damage.head()

,tower_damage,match_id,account_id,player_slot
0,14054,3574758829,132851371,1
1,13820,3574591160,91406909,2
2,12647,3574751256,101446572,128
3,11195,3574906857,96189126,132
4,11045,3574654085,412753955,128


Посмотрим глазками на результат

In [21]:
best_match_id = df_best_tower_damage.match_id[0]
db_result_url = '{}{}'.format(dotabuff_match, best_match_id)
result_url = '{}{}'.format(opendota_match, best_match_id)
print(db_result_url)
print(result_url)

https://www.dotabuff.com/matches/3574758829
https://www.opendota.com/matches/3574758829
